# Representation Learning - Part 1 - CIAFAR-10 Training

This notebook will detail the steps taken to learn representations of CIFAR-10 classes with two distinct data sets.

Eventhough there is one class contained in CIFAR-1o that is not in the QuickDraw set (the 'deer' class), we will use all 10 classes. THis will be done in the hopes that more training data may make the network trained on CIFAR-10 learn useful features that help to separate the 9 classes that do have an equivalent in QuickDraw.

We will now borrow heavily from the keras load_data() method from the cifar10 datasets module and the get_file() method from the data_utils module.

## V1 - Keras CNN with Dropout

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Use the same model as in the cifar10 CNN example from keras.

In [2]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

Define out image processing flow.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# we will do some mild image pre-processing for augmentation
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Add some callbacks, compile and get the training started.

In [5]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


batch_size = 32
epochs = 200

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath = 'v1-weights.{epoch:02d}-{val_loss:.4f}.hdf5'
model_chk = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=5, min_lr=0.001)

csv_log = CSVLogger('v1-training.log')

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr, model_chk, csv_log])

Epoch 1/200
1562/1562 [==============================] - 156s - loss: 1.8752 - acc: 0.3084 - val_loss: 1.5989 - val_acc: 0.4170
Epoch 2/200
1562/1562 [==============================] - 148s - loss: 1.6115 - acc: 0.4084 - val_loss: 1.3964 - val_acc: 0.4985
Epoch 3/200
1562/1562 [==============================] - 145s - loss: 1.4819 - acc: 0.4605 - val_loss: 1.3021 - val_acc: 0.5360
Epoch 4/200
1562/1562 [==============================] - 144s - loss: 1.3979 - acc: 0.4962 - val_loss: 1.2167 - val_acc: 0.5698
Epoch 5/200
1562/1562 [==============================] - 144s - loss: 1.3299 - acc: 0.5205 - val_loss: 1.1594 - val_acc: 0.5901
Epoch 6/200
1562/1562 [==============================] - 145s - loss: 1.2721 - acc: 0.5456 - val_loss: 1.1138 - val_acc: 0.6039
Epoch 7/200
1562/1562 [==============================] - 141s - loss: 1.2261 - acc: 0.5650 - val_loss: 1.0454 - val_acc: 0.6308
Epoch 8/200
1562/1562 [==============================] - 141s - loss: 1.1894 - acc: 0.5769 - val_loss: 1

1562/1562 [==============================] - 136s - loss: 0.8511 - acc: 0.7116 - val_loss: 0.6840 - val_acc: 0.7758
Epoch 65/200
1562/1562 [==============================] - 137s - loss: 0.8517 - acc: 0.7144 - val_loss: 0.6902 - val_acc: 0.7792
Epoch 66/200
1562/1562 [==============================] - 137s - loss: 0.8538 - acc: 0.7115 - val_loss: 0.6658 - val_acc: 0.7736
Epoch 67/200
1562/1562 [==============================] - 137s - loss: 0.8522 - acc: 0.7157 - val_loss: 0.6725 - val_acc: 0.7756
Epoch 68/200
1562/1562 [==============================] - 137s - loss: 0.8481 - acc: 0.7146 - val_loss: 0.7097 - val_acc: 0.7667
Epoch 69/200
1562/1562 [==============================] - 137s - loss: 0.8535 - acc: 0.7117 - val_loss: 0.6665 - val_acc: 0.7833
Epoch 70/200
1562/1562 [==============================] - 137s - loss: 0.8538 - acc: 0.7127 - val_loss: 0.6770 - val_acc: 0.7728
Epoch 71/200
1562/1562 [==============================] - 137s - loss: 0.8484 - acc: 0.7152 - val_loss: 0.6737

1562/1562 [==============================] - 137s - loss: 0.9772 - acc: 0.6822 - val_loss: 0.7842 - val_acc: 0.7493
Epoch 128/200
1562/1562 [==============================] - 137s - loss: 0.9706 - acc: 0.6836 - val_loss: 0.7820 - val_acc: 0.7555
Epoch 129/200
1562/1562 [==============================] - 137s - loss: 0.9875 - acc: 0.6763 - val_loss: 0.8586 - val_acc: 0.7340
Epoch 130/200
1562/1562 [==============================] - 137s - loss: 0.9869 - acc: 0.6762 - val_loss: 0.8469 - val_acc: 0.7480
Epoch 131/200
1562/1562 [==============================] - 137s - loss: 0.9955 - acc: 0.6755 - val_loss: 0.7354 - val_acc: 0.7542
Epoch 132/200
1562/1562 [==============================] - 137s - loss: 0.9994 - acc: 0.6757 - val_loss: 0.7806 - val_acc: 0.7458
Epoch 133/200
1562/1562 [==============================] - 137s - loss: 1.0086 - acc: 0.6716 - val_loss: 0.8014 - val_acc: 0.7295
Epoch 134/200
1562/1562 [==============================] - 137s - loss: 1.0210 - acc: 0.6673 - val_loss:

1562/1562 [==============================] - 137s - loss: 1.3503 - acc: 0.5613 - val_loss: 1.3844 - val_acc: 0.5768
Epoch 191/200
1562/1562 [==============================] - 137s - loss: 1.3531 - acc: 0.5624 - val_loss: 1.2280 - val_acc: 0.6183
Epoch 192/200
1562/1562 [==============================] - 137s - loss: 1.3563 - acc: 0.5601 - val_loss: 1.2126 - val_acc: 0.6014
Epoch 193/200
1562/1562 [==============================] - 137s - loss: 1.3670 - acc: 0.5598 - val_loss: 1.1747 - val_acc: 0.6443
Epoch 194/200
1562/1562 [==============================] - 137s - loss: 1.3625 - acc: 0.5544 - val_loss: 1.3267 - val_acc: 0.5606
Epoch 195/200
1562/1562 [==============================] - 137s - loss: 1.3804 - acc: 0.5532 - val_loss: 1.3706 - val_acc: 0.5485
Epoch 196/200
1562/1562 [==============================] - 137s - loss: 1.3722 - acc: 0.5542 - val_loss: 1.3738 - val_acc: 0.5498
Epoch 197/200
1562/1562 [==============================] - 137s - loss: 1.3673 - acc: 0.5559 - val_loss:

## V3 - FoDL CNN with Batch Normalization and Dropout

Network design taken from Fundamentals of Deep Learning by Nikhil Buduma (O'Reilly).

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Use the same model as in the cifar10 CNN example from keras.

In [2]:
# since we are using BatchNormalization in TensorFlow, we set this the axis to 3
bn_axis = 3

model = Sequential()

model.add(Conv2D(64, (5, 5), padding='same', input_shape=x_train.shape[1:], name='conv1'))
model.add(BatchNormalization(axis=bn_axis, name='bn_conv1'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), padding='same', name='conv2'))
model.add(BatchNormalization(axis=bn_axis, name='bn_conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(384, name='fc1'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(192, name='fc2'))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Define out image processing flow.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# we will do some mild image pre-processing for augmentation
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Add some callbacks, compile and get the training started.

In [4]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


batch_size = 32
epochs = 100

# initiate Adam opt1mizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath = 'v3-weights.{epoch:02d}-{val_loss:.4f}.hdf5'
model_chk = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

csv_log = CSVLogger('v3-training.log')

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[model_chk, csv_log])

Epoch 1/100
1562/1562 [==============================] - 331s - loss: 1.7229 - acc: 0.3748 - val_loss: 1.5415 - val_acc: 0.4311
Epoch 2/100
1562/1562 [==============================] - 311s - loss: 1.3985 - acc: 0.4947 - val_loss: 1.3988 - val_acc: 0.5075
Epoch 3/100
1562/1562 [==============================] - 317s - loss: 1.2788 - acc: 0.5455 - val_loss: 1.6261 - val_acc: 0.4446
Epoch 4/100
1562/1562 [==============================] - 327s - loss: 1.1849 - acc: 0.5836 - val_loss: 1.1321 - val_acc: 0.6154
Epoch 5/100
1562/1562 [==============================] - 322s - loss: 1.1265 - acc: 0.6070 - val_loss: 1.0465 - val_acc: 0.6258
Epoch 6/100
1562/1562 [==============================] - 315s - loss: 1.0789 - acc: 0.6217 - val_loss: 0.9948 - val_acc: 0.6486
Epoch 7/100
1562/1562 [==============================] - 327s - loss: 1.0339 - acc: 0.6417 - val_loss: 1.0292 - val_acc: 0.6371
Epoch 8/100
1562/1562 [==============================] - 327s - loss: 1.0050 - acc: 0.6517 - val_loss: 1

1562/1562 [==============================] - 296s - loss: 0.6126 - acc: 0.7915 - val_loss: 0.5458 - val_acc: 0.8139
Epoch 65/100
1562/1562 [==============================] - 296s - loss: 0.6155 - acc: 0.7897 - val_loss: 0.5860 - val_acc: 0.8005
Epoch 66/100
1562/1562 [==============================] - 296s - loss: 0.6102 - acc: 0.7916 - val_loss: 0.5989 - val_acc: 0.7989
Epoch 67/100
1562/1562 [==============================] - 295s - loss: 0.6060 - acc: 0.7925 - val_loss: 0.5283 - val_acc: 0.8227
Epoch 68/100
1562/1562 [==============================] - 296s - loss: 0.6093 - acc: 0.7928 - val_loss: 0.5747 - val_acc: 0.8072
Epoch 69/100
1562/1562 [==============================] - 296s - loss: 0.6062 - acc: 0.7919 - val_loss: 0.5581 - val_acc: 0.8121
Epoch 70/100
1562/1562 [==============================] - 296s - loss: 0.5980 - acc: 0.7967 - val_loss: 0.4917 - val_acc: 0.8328
Epoch 71/100
1562/1562 [==============================] - 296s - loss: 0.6028 - acc: 0.7959 - val_loss: 0.5657

## V4 - FoDL CNN with Batch Normalization Only

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Use the same model as in the cifar10 CNN example from keras.

In [2]:
# since we are using BatchNormalization in TensorFlow, we set this the axis to 3
bn_axis = 3

model = Sequential()

model.add(Conv2D(64, (5, 5), padding='same', input_shape=x_train.shape[1:], name='conv1'))
model.add(BatchNormalization(axis=3, name='bn_conv1'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), padding='same', name='conv2'))
model.add(BatchNormalization(axis=3, name='bn_conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(384, name='fc1'))
model.add(BatchNormalization(axis=1, name='bn_fc1'))
model.add(Activation('relu'))
model.add(Dense(192, name='fc2'))
model.add(BatchNormalization(axis=1, name='bn_fc2'))
model.add(Activation('relu'))
model.add(Dense(num_classes, name='output'))
model.add(BatchNormalization(axis=1, name='bn_outptu'))
model.add(Activation('softmax'))

Define out image processing flow.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# we will do some mild image pre-processing for augmentation
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Add some callbacks, compile and get the training started.

In [4]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


batch_size = 32
epochs = 100

# initiate Adam opt1mizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath = 'v4-weights.{epoch:02d}-{val_loss:.4f}.hdf5'
model_chk = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

csv_log = CSVLogger('v4-training.log')

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[model_chk, csv_log])

Epoch 1/100
1562/1562 [==============================] - 332s - loss: 1.4255 - acc: 0.5007 - val_loss: 1.5459 - val_acc: 0.4638
Epoch 2/100
1562/1562 [==============================] - 331s - loss: 1.1445 - acc: 0.6004 - val_loss: 1.3762 - val_acc: 0.5227: 5s - 
Epoch 3/100
1562/1562 [==============================] - 329s - loss: 1.0280 - acc: 0.6424 - val_loss: 1.4590 - val_acc: 0.4983
Epoch 4/100
1562/1562 [==============================] - 308s - loss: 0.9427 - acc: 0.6714 - val_loss: 0.8671 - val_acc: 0.7002
Epoch 5/100
1562/1562 [==============================] - 295s - loss: 0.8862 - acc: 0.6917 - val_loss: 0.9696 - val_acc: 0.6601
Epoch 6/100
1562/1562 [==============================] - 317s - loss: 0.8355 - acc: 0.7114 - val_loss: 0.7672 - val_acc: 0.7377
Epoch 7/100
1562/1562 [==============================] - 326s - loss: 0.7961 - acc: 0.7243 - val_loss: 0.7060 - val_acc: 0.7580
Epoch 8/100
1562/1562 [==============================] - 328s - loss: 0.7677 - acc: 0.7333 - val_

1562/1562 [==============================] - 299s - loss: 0.3219 - acc: 0.8881 - val_loss: 0.5520 - val_acc: 0.8246
Epoch 64/100
1562/1562 [==============================] - 295s - loss: 0.3151 - acc: 0.8900 - val_loss: 0.5272 - val_acc: 0.8373
Epoch 65/100
1562/1562 [==============================] - 295s - loss: 0.3171 - acc: 0.8907 - val_loss: 0.5137 - val_acc: 0.8430
Epoch 66/100
1562/1562 [==============================] - 295s - loss: 0.3127 - acc: 0.8929 - val_loss: 0.5082 - val_acc: 0.8393
Epoch 67/100
1562/1562 [==============================] - 295s - loss: 0.3112 - acc: 0.8926 - val_loss: 0.6278 - val_acc: 0.8162
Epoch 68/100
1562/1562 [==============================] - 295s - loss: 0.3080 - acc: 0.8933 - val_loss: 0.5946 - val_acc: 0.8120
Epoch 69/100
1562/1562 [==============================] - 295s - loss: 0.2969 - acc: 0.8982 - val_loss: 0.5308 - val_acc: 0.8312
Epoch 70/100
1562/1562 [==============================] - 294s - loss: 0.2966 - acc: 0.8981 - val_loss: 0.5300

## V5 - Keras CNN with Batch Normalization Only

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Use the same model as in the cifar10 CNN example from keras.

In [2]:
# since we are using BatchNormalization in TensorFlow, we set this the axis to 3
bn_axis = 3

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], name='conv1'))
model.add(BatchNormalization(axis=3, name='bn_conv1'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), name='conv2'))
model.add(BatchNormalization(axis=3, name='bn_conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', name='conv3'))
model.add(BatchNormalization(axis=3, name='bn_conv3'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), name='conv4'))
model.add(BatchNormalization(axis=3, name='bn_conv4'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, name='fc1'))
model.add(BatchNormalization(axis=1, name='bn_fc1'))
model.add(Activation('relu'))
model.add(Dense(num_classes, name='output'))
model.add(BatchNormalization(axis=1, name='bn_outptut'))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 32)        896       
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 30, 30, 32)        9248      
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 30, 30, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
__________

Define out image processing flow.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# we will do some mild image pre-processing for augmentation
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Add some callbacks, compile and get the training started.

In [4]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


batch_size = 32
epochs = 100

# initiate Adam opt1mizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath = 'v5-weights.{epoch:02d}-{val_loss:.4f}.hdf5'
model_chk = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

csv_log = CSVLogger('v5-training.log')

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[model_chk, csv_log])

Epoch 1/100
1562/1562 [==============================] - 273s - loss: 1.3519 - acc: 0.5293 - val_loss: 1.3831 - val_acc: 0.5283
Epoch 2/100
1562/1562 [==============================] - 245s - loss: 1.0370 - acc: 0.6445 - val_loss: 1.2745 - val_acc: 0.5644
Epoch 3/100
1562/1562 [==============================] - 244s - loss: 0.9046 - acc: 0.6917 - val_loss: 0.9614 - val_acc: 0.6636
Epoch 4/100
1562/1562 [==============================] - 241s - loss: 0.8341 - acc: 0.7132 - val_loss: 0.7704 - val_acc: 0.7340
Epoch 5/100
1562/1562 [==============================] - 242s - loss: 0.7741 - acc: 0.7339 - val_loss: 1.1966 - val_acc: 0.6106
Epoch 6/100
1562/1562 [==============================] - 245s - loss: 0.7326 - acc: 0.7472 - val_loss: 0.7136 - val_acc: 0.7553
Epoch 7/100
1562/1562 [==============================] - 246s - loss: 0.6984 - acc: 0.7583 - val_loss: 0.6204 - val_acc: 0.7886
Epoch 8/100
1562/1562 [==============================] - 247s - loss: 0.6697 - acc: 0.7692 - val_loss: 0

1562/1562 [==============================] - 239s - loss: 0.2783 - acc: 0.9031 - val_loss: 0.3822 - val_acc: 0.8771
Epoch 65/100
1562/1562 [==============================] - 239s - loss: 0.2791 - acc: 0.9042 - val_loss: 0.4253 - val_acc: 0.8651
Epoch 66/100
1562/1562 [==============================] - 246s - loss: 0.2736 - acc: 0.9058 - val_loss: 0.4204 - val_acc: 0.8679
Epoch 67/100
1562/1562 [==============================] - 244s - loss: 0.2735 - acc: 0.9054 - val_loss: 0.4113 - val_acc: 0.8669
Epoch 68/100
1562/1562 [==============================] - 240s - loss: 0.2675 - acc: 0.9067 - val_loss: 0.3861 - val_acc: 0.8770
Epoch 69/100
1562/1562 [==============================] - 240s - loss: 0.2655 - acc: 0.9083 - val_loss: 0.3790 - val_acc: 0.8805
Epoch 70/100
1562/1562 [==============================] - 251s - loss: 0.2641 - acc: 0.9081 - val_loss: 0.4288 - val_acc: 0.8645
Epoch 71/100
1562/1562 [==============================] - 256s - loss: 0.2604 - acc: 0.9103 - val_loss: 0.4214

## V5 - A - Keras CNN with Batch Normalization Only

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Use the same model as in the cifar10 CNN example from keras.

In [2]:
# since we are using BatchNormalization in TensorFlow, we set this the axis to 3
bn_axis = 3

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], name='conv1'))
model.add(BatchNormalization(axis=3, name='bn_conv1'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), name='conv2'))
model.add(BatchNormalization(axis=3, name='bn_conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', name='conv3'))
model.add(BatchNormalization(axis=3, name='bn_conv3'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), name='conv4'))
model.add(BatchNormalization(axis=3, name='bn_conv4'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, name='fc1'))
model.add(BatchNormalization(axis=1, name='bn_fc1'))
model.add(Activation('relu'))
model.add(Dense(num_classes, name='output'))
model.add(BatchNormalization(axis=1, name='bn_outptut'))
model.add(Activation('softmax'))

Define out image processing flow.

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# we will do some mild image pre-processing for augmentation
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Add some callbacks, compile and get the training started.

In [4]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


batch_size = 32
epochs = 150

# initiate Adam opt1mizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath = 'v5a-weights.{epoch:02d}-{val_loss:.4f}.hdf5'
model_chk = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

csv_log = CSVLogger('v5a-training.log')

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[model_chk, csv_log])

Epoch 1/150
1562/1562 [==============================] - 274s - loss: 1.3457 - acc: 0.5337 - val_loss: 1.2060 - val_acc: 0.5939
Epoch 2/150
1562/1562 [==============================] - 272s - loss: 1.0363 - acc: 0.6464 - val_loss: 0.9069 - val_acc: 0.6919
Epoch 3/150
1562/1562 [==============================] - 260s - loss: 0.9051 - acc: 0.6895 - val_loss: 0.8789 - val_acc: 0.6956
Epoch 4/150
1562/1562 [==============================] - 270s - loss: 0.8403 - acc: 0.7110 - val_loss: 0.7062 - val_acc: 0.7575
Epoch 5/150
1562/1562 [==============================] - 267s - loss: 0.7802 - acc: 0.7323 - val_loss: 0.6751 - val_acc: 0.7740
Epoch 6/150
1562/1562 [==============================] - 267s - loss: 0.7403 - acc: 0.7462 - val_loss: 0.6216 - val_acc: 0.7847
Epoch 7/150
1562/1562 [==============================] - 267s - loss: 0.6977 - acc: 0.7594 - val_loss: 0.6294 - val_acc: 0.7817
Epoch 8/150
1562/1562 [==============================] - 267s - loss: 0.6738 - acc: 0.7676 - val_loss: 0

1562/1562 [==============================] - 239s - loss: 0.2799 - acc: 0.9018 - val_loss: 0.4106 - val_acc: 0.8664
Epoch 65/150
1562/1562 [==============================] - 239s - loss: 0.2841 - acc: 0.9028 - val_loss: 0.4345 - val_acc: 0.8586
Epoch 66/150
1562/1562 [==============================] - 239s - loss: 0.2749 - acc: 0.9050 - val_loss: 0.3923 - val_acc: 0.8727
Epoch 67/150
1562/1562 [==============================] - 239s - loss: 0.2699 - acc: 0.9060 - val_loss: 0.4109 - val_acc: 0.8738
Epoch 68/150
1562/1562 [==============================] - 239s - loss: 0.2735 - acc: 0.9058 - val_loss: 0.4061 - val_acc: 0.8725
Epoch 69/150
1562/1562 [==============================] - 239s - loss: 0.2679 - acc: 0.9087 - val_loss: 0.4352 - val_acc: 0.8653
Epoch 70/150
1562/1562 [==============================] - 239s - loss: 0.2715 - acc: 0.9058 - val_loss: 0.3955 - val_acc: 0.8749
Epoch 71/150
1562/1562 [==============================] - 239s - loss: 0.2618 - acc: 0.9102 - val_loss: 0.4175

1562/1562 [==============================] - 239s - loss: 0.1889 - acc: 0.9346 - val_loss: 0.4664 - val_acc: 0.8596
Epoch 128/150
1562/1562 [==============================] - 239s - loss: 0.1920 - acc: 0.9332 - val_loss: 0.4206 - val_acc: 0.8719
Epoch 129/150
1562/1562 [==============================] - 239s - loss: 0.1936 - acc: 0.9326 - val_loss: 0.4310 - val_acc: 0.8655
Epoch 130/150
1562/1562 [==============================] - 239s - loss: 0.1919 - acc: 0.9332 - val_loss: 0.4173 - val_acc: 0.8760
Epoch 131/150
1562/1562 [==============================] - 239s - loss: 0.1922 - acc: 0.9327 - val_loss: 0.4122 - val_acc: 0.8744
Epoch 132/150
1562/1562 [==============================] - 239s - loss: 0.1888 - acc: 0.9350 - val_loss: 0.4197 - val_acc: 0.8733
Epoch 133/150
1562/1562 [==============================] - 239s - loss: 0.1877 - acc: 0.9349 - val_loss: 0.4236 - val_acc: 0.8693
Epoch 134/150
1562/1562 [==============================] - 240s - loss: 0.1867 - acc: 0.9354 - val_loss: